In [ ]:
!pip install pixiedust

In [ ]:
!pip install pixiedust_node

In [ ]:
import pixiedust_node

In [ ]:
# prerequisite - uncomment this to get the necessary node libraries used by this script
# npm.install("moment");
# npm.install("lodash");
# npm.install("mathjs");

In [ ]:
%%node

var moment = require('moment');
var mathjs = require('mathjs');
var _ = require('lodash');

In [ ]:
%%node 

function getTimeBucket(date1, date2) {
    var theDate = date1.getTime() < date2.getTime() ? date1 : date2;    
    return moment(theDate).startOf('day');
}

function typeHeartData(data) {
    const obj = {
        start: new Date(data[0]),
        finish: new Date(data[1]),
        count: Math.floor(+data[2]),
        day: null
    };

    obj.day = getTimeBucket(obj.start, obj.finish);

    return obj;
}

In [ ]:
%%node

var projectDataDir = `${process.env.HEALTH_ANALYSIS_ROOT}/data/heart`.replace(/\\/g, '/');
var heartDataPath = `${projectDataDir}/heart rate.csv`;
var heartSampleDataPath = `${projectDataDir}/heart rate sample.csv`;

In [ ]:
%%node

// blocking reads seems to be the fastest, using an event based non-blocking seems to overwhelm the runtime somehow, 
// too many queue'd callbacks to process for each row?
var hd = _.chain(fs.readFileSync(heartDataPath, 'utf-8').split('\n')).
    tail().
    map(r => r.split(',')).
    map(d => typeHeartData(d)).
    value();
    
print(`completed: hd size is: ${hd.length}`);

In [ ]:
%%node

var hd2017 = _.chain(hd).
    filter(data => data.day.year() === 2017).
    value();
    
var hd2018 = _.chain(hd).
    filter(data => data.day.year() === 2018).
    value();

In [ ]:
%%node

// display(hd2017);
display(_.sampleSize(hd2018, 20));

In [ ]:
%%node

var dayVar = _.chain(hd2018).
    groupBy(d => d.day).
    map((values, key) => ({ day: moment(key).format('YYYY-MM-DD'), dayVar: mathjs.var(values.map(v => v.count)) })).
    value();
    
// display(_.sampleSize(dayVar, 20));


In [ ]:
%%node
// display(_.sampleSize(dayVar, 20));
display(dayVar);
